In [ ]:
import torch.nn as nn
import numpy as np
import torch
import open3d as o3d
import matplotlib.pyplot as plt
import cv2


def euler2mat(angle):
    """Convert euler angles to rotation matrix.
     :param angle: [3] or [b, 3]
     :return
        rotmat: [3, 3] or [b, 3, 3]
    source
    https://github.com/ClementPinard/SfmLearner-Pytorch/blob/master/inverse_warp.py
    """

    if len(angle.size()) == 1:
        x, y, z = angle[0], angle[1], angle[2]
        _dim = 0
        _view = [3, 3]
    elif len(angle.size()) == 2:
        b, _ = angle.size()
        x, y, z = angle[:, 0], angle[:, 1], angle[:, 2]
        _dim = 1
        _view = [b, 3, 3]

    else:
        assert False

    cosz = torch.cos(z)
    sinz = torch.sin(z)

    # zero = torch.zeros([b], requires_grad=False, device=angle.device)[0]
    # one = torch.ones([b], requires_grad=False, device=angle.device)[0]
    zero = z.detach()*0
    one = zero.detach()+1
    zmat = torch.stack([cosz, -sinz, zero,
                        sinz, cosz, zero,
                        zero, zero, one], dim=_dim).reshape(_view)

    cosy = torch.cos(y)
    siny = torch.sin(y)

    ymat = torch.stack([cosy, zero, siny,
                        zero, one, zero,
                        -siny, zero, cosy], dim=_dim).reshape(_view)

    cosx = torch.cos(x)
    sinx = torch.sin(x)

    xmat = torch.stack([one, zero, zero,
                        zero, cosx, -sinx,
                        zero, sinx, cosx], dim=_dim).reshape(_view)

    rot_mat = xmat @ ymat @ zmat
    # print(rot_mat)
    return rot_mat

def point_transform(points, rot_mat, translation):
        """
        :param points: [batch, num_points, 3]
        :param rot_mat: [batch, 3]
        :param translation: [batch, 1, 3]
        :return:
        """
        rot_mat = rot_mat.to(points.device)
        translation = translation.to(points.device)
        points = torch.matmul(points, rot_mat)
        points = points - translation
        return points

def get_bbox(path):
    instance_bboxes = np.load(path)
    target_bboxes = np.zeros((64, 6))
    target_bboxes[0: instance_bboxes.shape[0], :] = instance_bboxes[:, 0:6]
    gt_centers = target_bboxes[:, 0:3]
    gt_centers[instance_bboxes.shape[0]:, :] += 1000.0
    size_gts = np.zeros((64, 3))
    size_gts[0:instance_bboxes.shape[0], :] = target_bboxes[0:instance_bboxes.shape[0], 3:6]
    
    return gt_centers, size_gts

def get_bbox_2d(gt_centers, size_gts, x_min, y_min, scale):
    centers = gt_centers[:, :, :2] # [B, 64, 2]
    sizes = size_gts['size_gts'][:, :, :2]
    centers[:, :, 0] = (centers[:, :, 0] - x_min) * scale
    centers[:, :, 1] = (centers[:, :, 1] - y_min) * scale
    sizes = sizes * scale.unsqueeze(-1) # [B, 64, 2]
    box2d = torch.cat([centers, sizes], dim=2)
    
    return box2d

def points2img(points_, colors, views_, image_height, image_width, size_x=4, size_y=4, return_info=False):
    """ Points to Image

    Args:
        points (Torch.Tensor): [B, num_points, 3]
        colors (Torch.Tensor): [B, num_points, 3] or None
        image_height (int): 
        image_width (int): 
        size_x (int, optional): _description_. Defaults to 4.
        size_y (int, optional): _description_. Defaults to 4.
        
    Return:
        imgs (Torch.Tensor): [B, image_height, image_width, 3]
    """
    B, N, _ = points_.shape
    
    v = 1
    angle = torch.tensor(views_[:, 0, :]).float().to(points_.device)
    rot_mat = euler2mat(angle).transpose(1, 2)
    
    translation = torch.tensor(views_[:, 1, :]).float().to(points_.device)
    translation = translation.unsqueeze(1)

    points = point_transform(points=torch.repeat_interleave(points_, v, dim=0),
            rot_mat=rot_mat.repeat(B, 1, 1),
            translation=translation.repeat(B, 1, 1))

    # print(points.shape)
    B = B*v
    colors = torch.repeat_interleave(colors, v, dim=0)
    
    assert size_x % 2 == 0 or size_x == 1
    assert size_y % 2 == 0 or size_y == 1
    
    
    coord_x = points[:, :, 0] - points[:, :, 0].min(dim=1)[0].unsqueeze(-1) # [batch, num_points]
    coord_y = points[:, :, 1] - points[:, :, 1].min(dim=1)[0].unsqueeze(-1) # [batch, num_points]

    scale = torch.min(torch.stack([(image_width-size_x)/coord_x.max(dim=1)[0].unsqueeze(-1), 
                          (image_height-size_y)/coord_y.max(dim=1)[0].unsqueeze(-1)], dim=0), dim=0)[0]
    
    coord_x = coord_x * scale + size_x/2
    coord_y = coord_y * scale + size_y/2
    
    _i = torch.linspace(-size_x/2, size_x/2-1, size_x, requires_grad=False, device=points.device)
    _j = torch.linspace(-size_y/2, size_y/2-1, size_y, requires_grad=False, device=points.device)
    
    extended_x = coord_x.unsqueeze(2).repeat([1, 1, size_x]) + _i # [batch, num_points, size_x]
    extended_y = coord_y.unsqueeze(2).repeat([1, 1, size_y]) + _j # [batch, num_points, size_y]
    extended_x = torch.clamp(extended_x, 0, image_width-1)
    extended_y = torch.clamp(extended_y, 0, image_height-1)
    
    extended_x = extended_x.ceil().long()
    extended_y = extended_y.ceil().long()
    
    extended_x = extended_x.unsqueeze(3).repeat([1, 1, 1, size_y]) # [batch, num_points, size_x, size_y]
    extended_y = extended_y.unsqueeze(2).repeat([1, 1, size_x, 1]) # [batch, num_points, size_x, size_y]
    
    # print(extended_x.min(), extended_x.max(), extended_y.min(), extended_y.max())
    # print(extended_x.shape, extended_y.shape)
    
    depth = points[:, :, 2]
    depth -= depth.min(dim=1)[0].unsqueeze(-1)
    depth /= depth.max(dim=1)[0].unsqueeze(-1)
    if colors is not None:
        if colors.max() > 1:
            colors = colors / 255
        value = colors.unsqueeze(3).unsqueeze(4).repeat(1, 1, 1, size_x, size_y).permute(0, 2, 1, 3, 4) # [batch, 3, num_points, size_x, size_y]
    else:
        value = depth.unsqueeze(1).unsqueeze(3).unsqueeze(4).repeat(1, 3, 1, size_x, size_y)  # [batch, 3, num_points, size_x, size_y]
    
    coordinates = extended_x.reshape([B, -1]) * image_width + extended_y.reshape([B, -1]) # [batch, num_points*size_x*size_y]
    coordinates = coordinates.unsqueeze(1).repeat(1, 3, 1) # [batch, 3, num_points*size_x*size_y]
    
    
    coordinates[:, 1, :] += image_height * image_width
    coordinates[:, 2, :] += image_height * image_width * 2
    coordinates = coordinates.reshape([B, -1]) # [batch, 3*num_points*size_x*size_y]
    
    value = value.reshape([B, -1])
    imgs = torch.zeros([B, 3*image_height*image_width], device='cpu')
    coordinates = coordinates.to('cpu')
    value = value.to('cpu')
    imgs = imgs.scatter_(1, coordinates.long(), value)
    imgs = imgs.to(points.device)
    
    imgs = imgs.reshape([B, 3, image_height, image_width])
    
    if not return_info:
        return imgs
    else:
        info = {}
        info['x_min'] = points[:, :, 0].min(dim=1)[0].unsqueeze(-1) # [batch, 1]
        info['y_min'] = points[:, :, 1].min(dim=1)[0].unsqueeze(-1) # [batch, 1]
        info['scale'] = scale
        info['rot_mat'] = rot_mat
        return imgs, info

    
# pcd = o3d.io.read_point_cloud("pc.ply")
# points = torch.tensor(np.asarray(pcd.points))
# points = points.unsqueeze(0).float().repeat(2, 1, 1)
# depth = points[0, :, 2]
# colors = torch.tensor(np.asarray(pcd.colors)).float().unsqueeze(0)
# colors = torch.stack([colors, depth.unsqueeze(-1).repeat(1, 1, 3)], dim=1).squeeze(0)
# print(colors.shape, points.shape)
# imgs = points2img(points, image_height=256, image_width=256, colors=colors, size_x=2, size_y=2)

import os
name = os.listdir('./scenes/')
name = [n for n in name if n.endswith('.npy')]
pts = []
cls = []
centers = []
sizes = []
for n in name:
    mesh_vertices = np.load(os.path.join('./scenes/', n))
    points = torch.tensor(mesh_vertices[:, 0:3])
    colors = torch.tensor(mesh_vertices[:, 3:6])
    # center, size = get_bbox(n.replace('vert', 'bbox')) 
    
    pts.append(points)
    cls.append(colors)
    # centers.append(center)
    # sizes.append(size)
points = torch.stack(pts, dim=0)
colors =  torch.stack(cls, dim=0)

# centers = torch.stack(centers, dim=0)
# sizes = torch.stack(sizes, dim=0)

views_ = np.asarray([
        [[-np.pi/2, 0, np.pi/6], [0, 0, 0]]
        ])
imgs, info = points2img(points, views_ = views_, image_height=256, image_width=256, colors=colors, size_x=2, size_y=2, return_info=True)

plt.rcParams['figure.figsize'] = (15, 15)

# plt.subplot(1, 2, 1)
# img = imgs[0].squeeze(0).detach().numpy().transpose(1, 2, 0)
# plt.imshow(img)
# plt.subplot(1, 2, 2)
# img = imgs[1].squeeze(0).detach().numpy().transpose(1, 2, 0)
idx = 2
img = imgs[idx].detach().numpy().transpose(1, 2, 0)
plt.imshow(img)
# plt.savefig("scene0306_01.png")
plt.show()

p = points[idx].detach().numpy()
c = colors[idx].detach().numpy()/255
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(p)
pcd.colors = o3d.utility.Vector3dVector(c)
o3d.visualization.draw_geometries([pcd])

In [4]:
import json
import numpy as np
t = json.load(open('t.json', 'r'))
a = []
for i in t:
    a.append(i['ref_on_img_max']/i['ref_points_num'])
a = np.asarray(a)
a.mean()

0.7598500756728087